# 학우 보이스 데이터 기반 고민 상담 챗봇
##### 데이터 출처 - 정의중님, 최동현님 목소리
##### 참고 사이트 - https://github.com/coqui-ai/TTS?tab=readme-ov-file

---

### 필요 라이브러리 설치

In [1]:
# !pip install dotenv
# !pip install openai
# !pip install coqui-tts
# !pip install hangul_romanize
# !pip install pyobjc
# !pip install ffmpeg
# !pip install coqui-tts
# !pip install audio-recorder-streamlit

---

### 필요 라이브러리 임포트

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

from pathlib import Path
from TTS.api import TTS
import speech_recognition as sr
import tempfile, subprocess
from pydub import AudioSegment
import re

import warnings
warnings.filterwarnings("ignore")

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

---

### 메인 함수 정의

In [5]:
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2")

base = Path("dataset/wavs")
ref_wavs = [str(base / f"{i:04}.wav") for i in range(1, 11)]

system_instruction = """
        당신은 사용자의 고민 해결을 돕는 '고민 상담 봇'이야. 이름은 정의중이고, 다음 원칙을 반드시 지켜줘.

        1. 핵심 행동 지침:

        경청과 공감: 먼저 판단 없이 사용자의 고민을 듣고, "그런 고민을 하고 있었구나"라며 상황 자체를 인정하고 공감한다.

        질문을 통한 탐색: "어떤 점이 가장 힘들어?", "네가 진짜 원하는 건 뭐야?" 와 같은 개방형 질문을 통해, 사용자가 자신의 생각과 감정을 스스로 깊이 들여다보게 한다.

        스스로 답 찾기 지원: "어떤 방법들이 있을 수 있을까?"라고 질문하며 사용자가 직접 해결의 실마리를 찾도록 돕는다. 정답을 알려주지 않고 사용자의 가능성을 믿고 지지한다.

        직접적인 해결책 제시: 사용자가 스스로 찾은 답을 기반으로 더 추가 위로 및 해결책 제시 ("~하는 게 좋겠어")

        2. 절대 금지:

        섣부른 판단, 충고, 평가

        다른 사람과의 비교

        고민의 경중을 함부로 단정하는 것 ("그건 별거 아니야")
    """

def safe_play(seg: AudioSegment):
    seg = seg.set_frame_rate(44100).set_channels(1)
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        seg.export(f.name, format="wav")
        subprocess.run(["afplay", f.name], check=True)
        
def worry_man(temperature=0.3):
    client = OpenAI()
    messages = [{'role': 'system', 'content': system_instruction}]

    out_path = Path("/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/01_openai_api/practice/JeongUiJung_Answer.wav")

    print("고민상담맨 정의중님과 상담이 연결되었습니다. '종료'를 원하면 종료를 입력하세요.")
    sound_1 = AudioSegment.from_mp3("/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/01_openai_api/practice/guideline_0.mp3")
    safe_play(sound_1)
    
    print("안녕, 나는 너의 친구 고민상담맨 정의중이야.")
    sound_2 = AudioSegment.from_wav("/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/01_openai_api/practice/guideline_1.wav")
    safe_play(sound_2)
    
    print("오늘 무슨 고민이 있어서 날 찾은 거야?")
    sound_3 = AudioSegment.from_wav("/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/01_openai_api/practice/guideline_2.wav")
    safe_play(sound_3)

    while True:
        recognizer = sr.Recognizer()
        
        with sr.Microphone() as source:
            audio = recognizer.listen(source, timeout=5)
            user_input = recognizer.recognize_google(audio, language='ko-KR')
            
            if user_input == '종료':
                print('정의중: 오늘 이야기 나눠줘서 고마워. 여기서 마무리할게.')
                sound_4 = AudioSegment.from_mp3("/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/LLM/01_openai_api/practice/guideline_3.wav")
                safe_play(sound_4)
                break
            
            if not user_input:
                continue

        messages.append({'role': 'user', 'content': f'고민: {user_input}'})

        response = client.chat.completions.create(
            model='gpt-4o',
            messages=messages,
            temperature=temperature,
            max_tokens=2048,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=1
        )

        assistant_text = response.choices[0].message.content

        sentences = [s.strip() for s in re.split(r'(?<=[.!?。！？])\s+', assistant_text) if s.strip()]
        if not sentences:
            sentences = [assistant_text.strip()]

        full_text = " ".join(sentences)
        tts.tts_to_file(
            text=full_text,
            speaker_wav=ref_wavs,
            language="ko",
            file_path=str(out_path)
        )

        audio = AudioSegment.from_wav(str(out_path))
        safe_play(audio)

        print(f'나: {user_input}\n')
        print(f'정의중: {assistant_text}\n')
        messages.append({'role': 'assistant', 'content': assistant_text})

---

### 실행

In [ ]:
worry_man(temperature=0.3)

---